<a href="https://colab.research.google.com/github/olaviinha/MIDIGenerators/blob/main/RandomArpeggioSequencer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Random Arpeggio Sequencer <font color="#999" size="3">v0.0.1</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/MidiGenerators" target="_blank"><font color="#999" size="4">Github</font></a><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font size="3" color="#999"><a href="https://inha.se" target="_blank"><font color="#999">O. Inha</font></a></font></font>

Random Arpeggio Sequencer does _almost_ that: generates randomized arpeggio-like (technically it's not really an arpeggio) sequence by given chord, bpm and other settings. It outputs a MIDI file, ready to be drag & dropped to your DAW.

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisities.

force_setup = False
pip_packages = 'mido pychord pyfluidsynth midi2audio'

import os
from google.colab import drive, output

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  %cd /content/
  !apt install fluidsynth
  output.clear()
  !pip -q install import-ipynb {pip_packages}
  output.clear()
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

tmp = '/content/tmp/'
create_dirs([tmp])

if not os.path.isdir('/content/sf') and force_setup == False:
  !gsutil -q -m cp -R gs://neural-research/olaviinha/sf/* {tmp}

output.clear()
op(c.ok, 'Setup finished.')


In [ ]:
#@markdown ##Basics
#@markdown <small>If you use multiple octaves, `root_octave` is the first and lowest octave.</small>
bpm = 124 #@param {type:"slider", min:60, max:200, step:1}
notes_per_beat = 4 #@param {type:"slider", min:1, max:8, step:1}

scale = 'A' #@param ["Ab", "A", "A#", "Bb", "B", "Cb", "C", "C#", "Db", "D", "D#", "Eb", "E", "F", "F#", "Gb", "G", "G#"]
quality = '9sus4' #@param ["---- 2 notes ----", "5", "sus", "---- 3 notes ----", "<none>", "maj", "m", "min", "-", "dim", "aug", "sus2", "sus4", "---- 4 notes ----", "6", "7", "7-5", "7b5", "7+5", "7#5", "7sus4", "m6", "m7", "m7-5", "m7b5", "dim6", "M7", "maj7", "M7+5", "mM7", "add9", "madd9", "2", "add11", "4", "---- 5 notes ----", "m69", "69", "9", "m9", "M9", "maj9", "9sus4", "7-9", "7b9", "7+9", "7#9", "9-5", "9b5", "9+5", "9#5", "7#9b5", "7#9#5", "7b9b5", "7b9#5", "11", "7+11", "7#11", "7b9#9", "7b9#11", "7#9#11", "7-13", "7b13", "---- 6 notes ----", "7b9b13", "9+11", "9#11", "13", "13-9", "13b9", "13+9", "13#9", "13+11", "13#11"]

root_octave = 3 #@param {type:"slider", min:1, max:8, step:1}
octaves = 2 #@param {type:"slider", min:1, max:4, step:1}
#change_octave = 'none' #@param ["none", "up", "down", "up-down", "down-up"]


#@markdown ##Note hits
velocity_min = 61 #@param {type:"slider", min:1, max:127, step:1}
velocity_max = 123 #@param {type:"slider", min:0, max:127, step:1}
## note_sustain = 128 #@param {type:"slider", min:1, max:128, step:1}

#@markdown ##Output
#@markdown <small>Output length in bars of 4 beats.</small>
output_length = 16 #@param {type:"slider", min:4, max:64, step:4}
preview_instrument = "jazz_guitar" #@param ["piano", "jazz_guitar", "organ", "rhodes", "tubularbell", "303", "sine_saw", "NES"]
#@markdown <small>Path to a directory located in your Google Drive, where all _.mid_ files will be saved.</small>
save_dir = "" #@param {type:"string"}

sf2dir = tmp+'base/'
if preview_instrument == 'piano': pi = sf2dir+'SalC5Light2.sf2'
if preview_instrument == 'jazz_guitar': pi = sf2dir+'inha-FatBoy-jazzguitar.sf2'
if preview_instrument == 'organ': pi = sf2dir+'inha-FatBoy-organ-rock.sf2'
if preview_instrument == 'rhodes': pi = sf2dir+'inha-FatBoy-rhodes.sf2'
if preview_instrument == 'tubularbell': pi = sf2dir+'inha-FatBoy-tubularbell.sf2'
if preview_instrument == '303': pi = sf2dir+'inha-HSTB303.sf2'
if preview_instrument == 'sine_saw': pi = sf2dir+'inha-nes-sinesaw.sf2'
if preview_instrument == 'nes': pi = sf2dir+'inha-nes-base.sf2'

if quality == '<none>':
  quality = ''

save_dir = fix_path(drive_root+save_dir)
beats_per_bar = 4

#-----------------------------------------------------------------#


debug = False

#---------------------------------------------------
#------- generate list of notes --------------------
import random
from itertools import groupby
from pychord import Chord

def generate_notes(chord='Amaj', octave=3, additional_octaves=1, notes_per_beat=1):
  global debug
  chord_notes = Chord(chord).components_with_pitch(root_pitch=octave)
  if additional_octaves > 1:
    for i in range(additional_octaves):
      if i > 0:
        chord_notes.extend(Chord(chord).components_with_pitch(root_pitch=octave+i))
  notes = chord_notes.copy()
  random.shuffle(notes)
  all_notes = []
  for _ in range(beats_per_bar * output_length):
    extended_notes = []
    for _ in range(8):
      extended_notes.extend(random.sample(notes.copy(), len(notes)))
    if debug == True: print('extended_notes:', extended_notes)

    nrpn = [i[0] for i in groupby(extended_notes)]
    beat_notes = nrpn[:notes_per_beat]
    if debug == True: print('beat_notes:', beat_notes)
    all_notes.extend(beat_notes)

  if debug == True: print('\nall_notes:', all_notes, '')

  #remove repeats from all_notes
  nrpn = [i[0] for i in groupby(all_notes.copy()*8)]
  final_notes = nrpn[:notes_per_beat*beats_per_bar*output_length]

  return final_notes

all_notes = generate_notes(chord=scale+quality, octave=root_octave, additional_octaves=octaves, notes_per_beat=notes_per_beat)
#------- //generate list of notes ------------------
#---------------------------------------------------


#---------------------------------------------------
#------- convert notes to midi ---------------------
midi_notes = []
for note in all_notes:
  midi_note = librosa.note_to_midi(note)
  midi_notes.append(midi_note)
if debug == True: print(midi_notes)
#------- //convert notes to midi -------------------
#---------------------------------------------------


#---------------------------------------------------
#------- generate midi file ------------------------
from random import randrange
from mido import Message, MidiFile, MidiTrack, MetaMessage
import mido

ticks_per_beat = 128

note_time = math.floor(ticks_per_beat/notes_per_beat)

mid = MidiFile(ticks_per_beat=ticks_per_beat)
track = MidiTrack()
mid.tracks.append(track)

track.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))
track.append(MetaMessage('set_tempo', tempo=mido.bpm2tempo(bpm), time=0))

for i, midi_note in enumerate(midi_notes):
  velocity = randrange(velocity_min, velocity_max)
  track.append(Message('note_on', note=midi_note, velocity=velocity, time=0))
  track.append(Message('note_off', note=midi_note, velocity=127, time=note_time))

if octaves == 1:
  octo = '_oct'+str(root_octave)
else:
  octo = '_oct'+str(root_octave)+'-'+str(root_octave+octaves-1)
midi_filename = save_dir+'ra_'+str(bpm)+'bpm_'+scale+quality+octo+'__'+rnd_str(4)+'.mid'
mid.save(midi_filename)

op(c.ok, 'File saved as:', midi_filename.replace(drive_root, ''))
print('')
#------- //generate midi file ----------------------
#---------------------------------------------------


#---------------------------------------------------
#------- generate audio preview --------------------
from midi2audio import FluidSynth
from IPython.display import display, Audio
audio_preview = 'output.wav'
FluidSynth(pi).midi_to_audio(midi_filename, audio_preview)
Audio(audio_preview)
#------- //generate audio preview ------------------
#---------------------------------------------------